In [1]:
! pip install -q sentence-transformers cohere

In [2]:
from sentence_transformers import CrossEncoder

In [3]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
top_4_documents = ['Efficient keyword extraction enhances search accuracy.',
 'Machine learning algorithms can optimize keyword extraction methods.',
 'Understanding document structure aids in keyword extraction.',
 'Document analysis involves extracting keywords.']

In [5]:
query = "Natural language processing techniques enhance keyword extraction efficiency."

In [6]:
pairs = []
for doc in top_4_documents:
  pairs.append([query,doc])

In [7]:
pairs

[['Natural language processing techniques enhance keyword extraction efficiency.',
  'Efficient keyword extraction enhances search accuracy.'],
 ['Natural language processing techniques enhance keyword extraction efficiency.',
  'Machine learning algorithms can optimize keyword extraction methods.'],
 ['Natural language processing techniques enhance keyword extraction efficiency.',
  'Understanding document structure aids in keyword extraction.'],
 ['Natural language processing techniques enhance keyword extraction efficiency.',
  'Document analysis involves extracting keywords.']]

In [8]:
scores = cross_encoder.predict(pairs)
scores

array([ 3.1378708 ,  0.84216565, -2.9193    , -2.878192  ], dtype=float32)

In [9]:
scored_docs = zip(scores,top_4_documents)
scored_docs

In [10]:
reranked_document_cross_encoder = sorted(scored_docs,reverse=True)
reranked_document_cross_encoder

[(3.1378708, 'Efficient keyword extraction enhances search accuracy.'),
 (0.84216565,
  'Machine learning algorithms can optimize keyword extraction methods.'),
 (-2.878192, 'Document analysis involves extracting keywords.'),
 (-2.9193, 'Understanding document structure aids in keyword extraction.')]

# Cohere

In [11]:
import cohere
from google.colab import userdata

In [12]:
COHERE_API = userdata.get('COHERE_API')

In [13]:
co = cohere.Client(COHERE_API)

In [14]:
top_4_documents

['Efficient keyword extraction enhances search accuracy.',
 'Machine learning algorithms can optimize keyword extraction methods.',
 'Understanding document structure aids in keyword extraction.',
 'Document analysis involves extracting keywords.']

In [15]:
query

'Natural language processing techniques enhance keyword extraction efficiency.'

In [16]:
response = co.rerank(
    model="rerank-english-v3.0",
    query="Natural language processing techniques enhance keyword extraction efficiency.",
    documents=top_4_documents,
    return_documents=True
)

In [17]:
response

RerankResponse(id='d94c863b-771f-4a9d-b454-1b178b7e0023', results=[RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='Efficient keyword extraction enhances search accuracy.'), index=0, relevance_score=0.99411184), RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='Machine learning algorithms can optimize keyword extraction methods.'), index=1, relevance_score=0.9129032), RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='Understanding document structure aids in keyword extraction.'), index=2, relevance_score=0.32885265), RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='Document analysis involves extracting keywords.'), index=3, relevance_score=0.02865267)], meta=ApiMeta(api_version=ApiMetaApiVersion(version='1', is_deprecated=None, is_experimental=None), billed_units=ApiMetaBilledUnits(images=None, input_tokens=None, output_tokens=None, search_units=1.0, classifications=None), tokens=None,

In [18]:
for i in range(4):
  print(f'text: {response.results[i].document.text} score: {response.results[i].relevance_score}')

text: Efficient keyword extraction enhances search accuracy. score: 0.99411184
text: Machine learning algorithms can optimize keyword extraction methods. score: 0.9129032
text: Understanding document structure aids in keyword extraction. score: 0.32885265
text: Document analysis involves extracting keywords. score: 0.02865267
